In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'ERLSENOGT)N AEA MLGASB/(AQKTDLCS ',
 'calculationPrice': 'close',
 'open': None,
 'openTime': None,
 'openSource': 'lcaofiif',
 'close': None,
 'closeTime': None,
 'closeSource': 'ialcfofi',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 128.29,
 'latestSource': 'Close',
 'latestTime': 'January 19, 2021',
 'latestUpdate': 1641506731824,
 'latestVolume': None,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 133.31,
 'previousVolume': 112472890,
 'change': 0.69,
 'changePercent': 0.00548,
 'volume': None,
 'iexMarketPercent': None,
 'iexVolume': None,
 'avgTotalVolume': 111445270,


In [4]:
pe_ratio = data['peRatio']
pe_ratio

40.16

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([
                                                            symbol,
                                                            data[symbol]['quote']['latestPrice'],
                                                            data[symbol]['quote']['peRatio'],
                                                            'N/A'
                                                            ],index = my_columns),
                                                            ignore_index = True)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,133.65,55.46,N/A
1,AAL,16.24,-1.19,N/A
2,AAP,167.78,25.06,N/A
3,AAPL,129.00,40.01,N/A
4,ABBV,115.26,24.94,N/A
...,...,...,...,...
500,YUM,108.32,31.26,N/A
501,ZBH,160.20,-219.5,N/A
502,ZBRA,409.28,46.33,N/A
503,ZION,51.42,21.27,N/A


In [9]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [11]:
portfolio_input()

Enter the value of your portfolio:10000000


In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,42.03,2.56,4758
1,NLOK,21.81,4.02,9170
2,AIV,4.89,4.84,40899
3,BIO,610.58,5.24,327
4,UNM,26.52,5.6,7541
5,AFL,48.58,7.6,4116
6,ALL,111.69,7.91,1790
7,CPB,48.39,8.04,4133
8,EBAY,58.21,8.24,3435
9,KIM,16.65,8.3,12012


In [13]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,219.16,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,238.64,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.74,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,32.96,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.25,N/A,14.92,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,55.35,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.92,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,159.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1849.56,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.06,N/A,32.22,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [18]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.865347
1       0.154455
2        0.49703
3       0.774257
4       0.506931
         ...    
500     0.633663
501    0.0257426
502     0.822772
503     0.439604
504     0.827723
Name: PE Percentile, Length: 505, dtype: object
0       0.746535
1      0.0475248
2       0.419802
3       0.962376
4       0.853465
         ...    
500    0.0455446
501     0.390099
502     0.837624
503     0.114851
504     0.928713
Name: PB Percentile, Length: 505, dtype: object
0       0.808911
1      0.0277228
2       0.165347
3       0.837624
4        0.69901
         ...    
500     0.740594
501     0.681188
502     0.708911
503     0.435644
504     0.934653
Name: PS Percentile, Length: 505, dtype: object
0      0.847525
1      0.029703
2      0.293069
3      0.792079
4      0.473267
         ...   
500    0.691089
501     0.80396
502    0.837624
503    0.328713
504    0.845545
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.815842
1      0.0534653
2       0.122772
3       0.934

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.71,N/A,55.62,0.865347,8.38,0.746535,7.3500,0.808911,33.984568,0.847525,14.287447,0.815842,N/A
1,AAL,16.26,N/A,-1.14,0.154455,-1.77,0.0475248,0.4091,0.0277228,-6.203538,0.029703,1.393223,0.0534653,N/A
2,AAP,169.28,N/A,24.89,0.49703,3.13,0.419802,1.1800,0.165347,11.825971,0.293069,2.644120,0.122772,N/A
3,AAPL,128.51,N/A,40.37,0.774257,34.26,0.962376,8.0900,0.837624,27.518752,0.792079,20.585908,0.934653,N/A
4,ABBV,114.66,N/A,25.42,0.506931,13.01,0.853465,5.0500,0.69901,16.167611,0.473267,9.636383,0.645545,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.44,N/A,31.33,0.633663,-4.18,0.0455446,5.8900,0.740594,22.790855,0.691089,10.071659,0.665347,N/A
501,ZBH,161.41,N/A,-219.85,0.0257426,2.84,0.390099,4.7700,0.681188,28.440019,0.80396,8.029821,0.512871,N/A
502,ZBRA,414.82,N/A,48.39,0.822772,11.34,0.837624,5.2200,0.708911,32.465209,0.837624,12.148634,0.750495,N/A
503,ZION,49.99,N/A,21.16,0.439604,1.18,0.114851,2.8100,0.435644,12.419399,0.328713,3.224106,0.162376,N/A


In [19]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.71,N/A,55.62,0.865347,8.38,0.746535,7.3500,0.808911,33.984568,0.847525,14.287447,0.815842,0.816832
1,AAL,16.26,N/A,-1.14,0.154455,-1.77,0.0475248,0.4091,0.0277228,-6.203538,0.029703,1.393223,0.0534653,0.0625743
2,AAP,169.28,N/A,24.89,0.49703,3.13,0.419802,1.1800,0.165347,11.825971,0.293069,2.644120,0.122772,0.299604
3,AAPL,128.51,N/A,40.37,0.774257,34.26,0.962376,8.0900,0.837624,27.518752,0.792079,20.585908,0.934653,0.860198
4,ABBV,114.66,N/A,25.42,0.506931,13.01,0.853465,5.0500,0.69901,16.167611,0.473267,9.636383,0.645545,0.635644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.44,N/A,31.33,0.633663,-4.18,0.0455446,5.8900,0.740594,22.790855,0.691089,10.071659,0.665347,0.555248
501,ZBH,161.41,N/A,-219.85,0.0257426,2.84,0.390099,4.7700,0.681188,28.440019,0.80396,8.029821,0.512871,0.482772
502,ZBRA,414.82,N/A,48.39,0.822772,11.34,0.837624,5.2200,0.708911,32.465209,0.837624,12.148634,0.750495,0.791485
503,ZION,49.99,N/A,21.16,0.439604,1.18,0.114851,2.8100,0.435644,12.419399,0.328713,3.224106,0.162376,0.296238


In [20]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [21]:
portfolio_input()

Enter the value of your portfolio:10000000


In [22]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

c:\users\pranit\anaconda3\envs\algotrade\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,F,10.32,19379,-246.58,0.0237624,1.2100,0.124752,0.3087,0.0118812,4.010659,0.039604,2.080082,0.0950495,0.0590099
1,AIG,43.38,4610,-7.68,0.110891,0.5973,0.0574257,0.8133,0.0772277,4.878120,0.0455446,0.802034,0.0178218,0.0617822
2,AAL,16.26,12300,-1.14,0.154455,-1.7700,0.0475248,0.4091,0.0277228,-6.203538,0.029703,1.393223,0.0534653,0.0625743
3,UNM,27.09,7382,5.78,0.19604,0.4880,0.0534653,0.4470,0.0336634,2.905180,0.0356436,0.430908,0.0039604,0.0645545
4,MET,53.42,3743,8.84,0.207921,0.6460,0.0613861,0.7130,0.0613861,4.894892,0.0475248,0.711832,0.0118812,0.0780198
5,BA,220.97,905,-27.40,0.0752475,-10.4500,0.0336634,2.0000,0.321782,-28.731929,0.0158416,-203.365573,0.0019802,0.089703
6,FTI,11.94,16750,-0.97,0.156436,1.2800,0.137624,0.3978,0.0257426,3.455347,0.0376238,2.317897,0.110891,0.0936634
7,ALL,113.58,1760,8.00,0.20099,1.3700,0.162376,0.7512,0.0633663,2.524014,0.0336634,0.761831,0.0138614,0.0948515
8,ABC,106.50,1877,-6.29,0.118812,-21.1200,0.019802,0.1114,0.0039604,8.764316,0.130693,3.999889,0.225743,0.099802
9,HIG,54.40,3676,11.38,0.233663,1.1200,0.105941,0.9262,0.0970297,4.608862,0.0415842,0.914411,0.0217822,0.1


In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [26]:
writer.save()